## Import necessary packages:

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, GridSearchCV
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import average_precision_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from tqdm import tqdm
from sklearn.pipeline import Pipeline

seed = 0

Probe = ['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan']

DoS = ['apache2', 'back', 'land', 'mailbomb', 'neptune', 'pod', 'processtable', 'smurf', 'teardrop', 'udpstorm']

R2L = ['buffer_overflow', 'httptunnel', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm']

U2R = ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'named', 'phf', 'sendmail', 'snmpgetattack', 'snmpguess',
       'spy', 'warezclient', 'warezmaster', 'worm', 'xlock', 'xsnoop']

## Import data and assign feature name to each column:

In [ ]:
columns = []
with open('kddcup.names') as f:
    next(f)
    for line in f:
        feature = line.split(':')[0]
        columns.append(feature)
columns.append('label')

In [ ]:
train_data = pd.read_csv('kddcup.data', header=None)
train_data.columns = columns
train_data['label'] = train_data['label'].apply(lambda x: x.replace('.', ''))

test_data = pd.read_csv('corrected', header=None)
test_data.columns = columns
test_data['label'] = test_data['label'].apply(lambda x: x.replace('.', ''))

In [ ]:
boundary = train_data.shape[0]
whole_data = pd.concat([train_data, test_data], axis=0)

## One-hot encoding:

In [ ]:
categorical = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']
whole_data_categorical = whole_data[categorical].copy()
whole_data.drop(columns=categorical, inplace=True)
whole_data_categorical_T = pd.get_dummies(whole_data_categorical)
whole_data = pd.concat([whole_data_categorical_T, whole_data], axis=1)

## Split data into training and test set:

In [ ]:
train_data = whole_data.iloc[:boundary]
test_data = whole_data.iloc[boundary:]

X_train, y_train = train_data.drop(columns='label'), train_data['label']
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed)
X_test, y_test = test_data.drop(columns='label'), test_data['label']

## Convert specific attack types to four general attack types in y_valid and y_test:

In [ ]:
for attack in Probe:
    y_valid = y_valid.replace(attack, 'Probe')
    y_test = y_test.replace(attack, 'Probe')
    
for attack in DoS:
    y_valid = y_valid.replace(attack, 'DoS')
    y_test = y_test.replace(attack, 'DoS')
    
for attack in R2L:
    y_valid = y_valid.replace(attack, 'R2L')
    y_test = y_test.replace(attack, 'R2L')
    
for attack in U2R:
    y_valid = y_valid.replace(attack, 'U2R')
    y_test = y_test.replace(attack, 'U2R')

## Store preprocessed data for later usage:

In [ ]:
X_train.to_pickle("X_train.pkl")
y_train.to_pickle("y_train.pkl")

X_valid.to_pickle("X_valid.pkl")
y_valid.to_pickle("y_valid.pkl")

X_test.to_pickle("X_test.pkl")
y_test.to_pickle("y_test.pkl")

## Load preprocessed data back:

In [ ]:
X_train = pd.read_pickle("X_train.pkl")
y_train = pd.read_pickle("y_train.pkl")

X_valid = pd.read_pickle("X_valid.pkl")
y_valid = pd.read_pickle("y_valid.pkl")

X_test = pd.read_pickle("X_test.pkl")
y_test = pd.read_pickle("y_test.pkl")

## Sample a subset of training data from the original training set:

In [ ]:
# Recombine X_train and y_train into a dataframe:
X_train_df = pd.concat([X_train, y_train], axis=1)

In [ ]:
# Divide instances by label:
label_list = np.unique(X_train_df['label'])

df_label_list_dict = {'Probe': [], 'DoS': [], 'R2L': [], 'U2R': [], 'normal': []}
for label in label_list:
    df_label = X_train_df[X_train_df['label'] == label]
    
    if label in Probe:
        df_label_list_dict['Probe'].append(df_label)
    elif label in DoS:
        df_label_list_dict['DoS'].append(df_label)
    elif label in R2L:
        df_label_list_dict['R2L'].append(df_label)
    elif label in U2R:
        df_label_list_dict['U2R'].append(df_label)
    else:
        df_label_list_dict['normal'].append(df_label)

In [ ]:
# Determine the maximum number of instances to be sampled from each label:
num_samples_per_label = 500

X_train_df_sampled = pd.DataFrame()
sample_weights = []

for cls, df_label_list in df_label_list_dict.items():
    sample_distribution_within_cls = []
    
    for df_label in df_label_list:
        num_instances = df_label.shape[0]
        num_samples = min(num_instances, num_samples_per_label)
        
        sample_distribution_within_cls.append(num_samples)
        df_sampled = df_label.sample(n=num_samples, random_state=seed)
        X_train_df_sampled = pd.concat([X_train_df_sampled, df_sampled], axis=0)
        
    sample_distribution_within_cls = np.array(sample_distribution_within_cls)
    weight_per_sample_group = sample_distribution_within_cls.sum() / sample_distribution_within_cls
    
    for i in range(len(sample_distribution_within_cls)):
        sample_weights += [weight_per_sample_group[i]] * sample_distribution_within_cls[i]

In [ ]:
# Decompose X_train_df_sampled into X_train and y_train again:
X_train, y_train = X_train_df_sampled.drop(columns='label'), X_train_df_sampled['label']

## Convert specific attack types to four general attack types in y_train:

In [ ]:
for attack in Probe:
    y_train = y_train.replace(attack, 'Probe')
    
for attack in DoS:
    y_train = y_train.replace(attack, 'DoS')
    
for attack in R2L:
    y_train = y_train.replace(attack, 'R2L')
    
for attack in U2R:
    y_train = y_train.replace(attack, 'U2R')

## Standardize feature vectors using StandardScaler:

In [ ]:
sscaler = preprocessing.StandardScaler().fit(X_train)
X_train, X_valid, X_test = sscaler.transform(X_train), sscaler.transform(X_valid), sscaler.transform(X_test)

## Use linear PCA to perform dimensionality reduction:
**Warning**: linear PCA **works** with StandardScaler but **doesn't work** with RobustScaler.

In [ ]:
pca = PCA(n_components=X_train.shape[1], svd_solver='full')
pca.fit(X_train)

fig, ax1 = plt.subplots(figsize=(8, 4))
graph1 = ax1.bar(np.arange(X_train.shape[1]), pca.explained_variance_ratio_, label='individual explained variance')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_xlabel('Number of Principal Components')
ax1.set_ylabel('Explained Variance Ratio')

ax2 = ax1.twinx()
graph2 = ax2.plot(np.cumsum(pca.explained_variance_ratio_), c='red', label='cumulative explained variance')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylabel('Explained Variance Ratio')

index = np.nonzero(np.cumsum(pca.explained_variance_ratio_) >= 0.9)[0][0]
graph3 = ax2.vlines(index, 0, 1, color='purple', linestyles="dotted", label='90% variance is explained with {} components'.format(index+1))

graphs = [graph1, graph2[0], graph3]
labels = [g.get_label() for g in graphs]
ax1.legend(graphs, labels, bbox_to_anchor=(1, 0.5))

plt.title('Explained Variance Ratio versus Number of Principal Components')
plt.show()

In [ ]:
# Define search region for linear PCA
components_search_linear = []

## Use Kernel PCA to perform dimensionality reduction

In [ ]:
pca = KernelPCA(n_components=X_train.shape[1], kernel='rbf')
pca.fit_transform(X_train)
variance_all = np.sum(pca.lambdas_)

fig, ax1 = plt.subplots(figsize=(8, 4))
graph1 = ax1.bar(np.arange(X_train.shape[1]), pca.lambdas_/variance_all, label='individual explained variance')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_xlabel('Number of Principal Components')
ax1.set_ylabel('Explained Variance Ratio')

ax2 = ax1.twinx()
graph2 = ax2.plot(np.cumsum(pca.lambdas_)/variance_all, c='red', label='cumulative explained variance')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylabel('Explained Variance Ratio')

index = np.nonzero(np.cumsum(pca.lambdas_)/variance_all >= 0.9)[0][0]
graph3 = ax2.vlines(index, 0, 1, color='purple', linestyles="dotted", label='90% variance is explained with {} components'.format(index+1))

graphs = [graph1, graph2[0], graph3]
labels = [g.get_label() for g in graphs]
ax1.legend(graphs, labels, bbox_to_anchor=(1, 0.5))

plt.title('Explained Variance Ratio versus Number of Principal Components')
plt.show()

In [ ]:
# Define search region for Kernel PCA
components_search_kernel = []

## Make custom scorers for grid search:

In [ ]:
def prob_avg_PR_scorer(clf, X, y):
    y_pred = clf.predict_proba(X)
    return average_precision_score(preprocessing.LabelBinarizer().fit_transform(y), y_pred, average='macro')

def deci_avg_PR_scorer(clf, X, y):
    y_pred = clf.decision_function(X)
    return average_precision_score(preprocessing.LabelBinarizer().fit_transform(y), y_pred, average='macro')

## Batch PCA. Aim to reduce memory usage

In [ ]:
def batchPCA(pca, data, batchSize):
    instance_num = data.shape[0]
    batch_num = instance_num // batchSize
    
    for batch_idx in tqdm(range(batch_num)):
        batch = data[0:batchSize]
        batch_pca = pca.transform(batch)
        if batch_idx == 0:
            pca_result = batch_pca
        else:
            pca_result = np.concatenate((pca_result, batch_pca), axis=0)
        data = np.delete(data, np.s_[0:batchSize], axis=0)

    batch_pca = pca.transform(data)
    pca_result = np.concatenate((pca_result, batch_pca), axis=0)  
    return pca_result

## Confusion matrix plot

In [ ]:
def confusion_matrx_plot(model, valid_data, valid_label):

    y_pred_valid = model.predict(valid_data)
    cm = confusion_matrix(valid_label, y_pred_valid)
    cm = pd.DataFrame(data=cm, index=[model.classes_], columns=[model.classes_])

    percent_correct_horizontal = cm.sum(axis=1)
    for i in range(cm.shape[0]):
        percent_correct_horizontal.iloc[i] = cm.iloc[i, i] / percent_correct_horizontal.iloc[i]
    percent_correct_horizontal = round(percent_correct_horizontal * 100, 2)
    percent_correct_horizontal = percent_correct_horizontal.apply(lambda x: str(x) + '%')

    percent_correct_vertical = cm.sum(axis=0)
    for i in range(cm.shape[0]):
        percent_correct_vertical.iloc[i] = cm.iloc[i, i] / percent_correct_vertical.iloc[i]
    percent_correct_vertical = round(percent_correct_vertical * 100, 2)
    percent_correct_vertical = percent_correct_vertical.apply(lambda x: str(x) + '%')

    cm['Recall%'] = percent_correct_horizontal
    percent_correct_vertical.name = ('Precision%', 5)
    cm = cm.append(percent_correct_vertical)
    
    display(cm)

## Stratified holdout validation:

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

## Stratified Cross Validation:

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Grid Search Session
## Linear PCA with Logistic Regression

In [ ]:
# Linear PCA with Logistic Regression

pipeline = Pipeline([
    ('pca', PCA(svd_solver='full')),
    ('model', LogisticRegression(multi_class='multinomial', class_weight='balanced', solver='lbfgs'))
])

param_grid = [
    {
        'pca__n_components': components_search_linear,
        'classify__C': np.logspace(-5, 5, 30)
    },
]

# Get best combination of pca and classifier
grid = GridSearchCV(pipeline, param_grid, prob_avg_PR_scorer, cv=skf)
grid.fit(X_train, y_train, sample_weights) # Note this part should use untransformed data
display(grid.best_params_)

best_combination = grid.best_estimatior # Best combination
best_pca = best_combination.named_steps['pca'] # Best PCA
best_clf = best_combination.named_steps['model'] # Best classifer

# Transform training data by selected PCA
best_pca.fit(X_train)
X_valid_tmp = batchPCA(best_pca, X_valid, 20000)

# Multiclass macro average PR_AUC:
avg_PR = prob_avg_PR_scorer(best_clf, X_valid_tmp, y_valid)

print ("The best average PR_AUC for Linear PCA with Logistic Regression on validation set is {:.4f}".format(avg_PR))
display(best_clf.best_params_)

confusion_matrx_plot(best_clf, X_valid_tmp, y_valid)

## Linear PCA with kernel SVM

In [ ]:
# Linear PCA with kernel PCA

pipeline = Pipeline([
    ('pca', PCA(svd_solver='full')),
    ('model', SVC(kernel='rbf', decision_function_shape='ovr', class_weight='balanced'))
])

param_grid = [
    {
        'pca__n_components': components_search_linear,
        'classify__C': np.logspace(-5, 5, 10),
        'classify__gamma': np.logspace(-5, 0, 5)
    },
]

# Get best combination of pca and classifier
grid = GridSearchCV(pipeline, param_grid, deci_avg_PR_scorer, cv=skf)
grid.fit(X_train, y_train, sample_weights) # Note this part should use untransformed data
display(grid.best_params_)

best_combination = grid.best_estimatior # Best combination
best_pca = best_combination.named_steps['pca'] # Best PCA
best_clf = best_combination.named_steps['model'] # Best classifer

# Transform training data by selected PCA
best_pca.fit(X_train)
X_valid_tmp = batchPCA(best_pca, X_valid, 20000)

# Multiclass macro average PR_AUC:
avg_PR = deci_avg_PR_scorer(best_clf, X_valid_tmp, y_valid)

print ("The best average PR_AUC for Linear PCA with Logistic Regression on validation set is {:.4f}".format(avg_PR))
display(best_clf.best_params_)

confusion_matrx_plot(best_clf, X_valid_tmp, y_valid)

## Kernel PCA with Logistic regression

In [ ]:
# Kernel PCA with Logistic Regression

pipeline = Pipeline([ 
    ('pca', KernelPCA(kernel='rbf')),
    ('model', LogisticRegression(multi_class='multinomial', class_weight='balanced', solver='lbfgs'))
])

param_grid = [
    {
        'pca__n_components': components_search_kernel,
        'pca__gamma': np.logspace(-5, 0, num=10),
        'classify__C': np.logspace(-5, 5, 30)
    },
]

# Get best combination of pca and classifier
grid = GridSearchCV(pipeline, param_grid, prob_avg_PR_scorer, cv=skf)
grid.fit(X_train, y_train, sample_weights) # Note this part should use untransformed data
display(grid.best_params_)

best_combination = grid.best_estimatior # Best combination
best_pca = best_combination.named_steps['pca'] # Best PCA
best_clf = best_combination.named_steps['model'] # Best classifer

# Transform training data by selected PCA
best_pca.fit(X_train)
X_valid_tmp = batchPCA(best_pca, X_valid, 20000)

# Multiclass macro average PR_AUC:
avg_PR = prob_avg_PR_scorer(best_clf, X_valid_tmp, y_valid)

print ("The best average PR_AUC for Linear PCA with Logistic Regression on validation set is {:.4f}".format(avg_PR))
display(best_clf.best_params_)

confusion_matrx_plot(best_clf, X_valid_tmp, y_valid)

## Kernel PCA with kernel SVM

In [ ]:
# Kernel PCA with kernel SVM

pipeline = Pipeline([    
    ('pca', KernelPCA(kernel='rbf')),
    ('model', LogisticRegression(multi_class='multinomial', class_weight='balanced', solver='lbfgs'))
])

param_grid = [
    {
        'pca__n_components': components_search_kernel,
        'pca__gamma': np.logspace(-5, 0, num=10),
        'classify__C': np.logspace(-5, 5, 10),
        'classify__gamma': np.logspace(-5, 0, 5)
    },
]

# Get best combination of pca and classifier
grid = GridSearchCV(pipeline, param_grid, deci_avg_PR_scorer, cv=skf)
grid.fit(X_train, y_train, sample_weights) # Note this part should use untransformed data
display(grid.best_params_)

best_combination = grid.best_estimatior # Best combination
best_pca = best_combination.named_steps['pca'] # Best PCA
best_clf = best_combination.named_steps['model'] # Best classifer

# Transform training data by selected PCA
best_pca.fit(X_train)
X_valid_tmp = batchPCA(best_pca, X_valid, 20000)

# Multiclass macro average PR_AUC:
avg_PR = deci_avg_PR_scorer(best_clf, X_valid_tmp, y_valid)

print ("The best average PR_AUC for Linear PCA with Logistic Regression on validation set is {:.4f}".format(avg_PR))
display(best_clf.best_params_)

confusion_matrx_plot(best_clf, X_valid_tmp, y_valid)